![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Hands-On Lab
# Welcome to the Understanding Server Configuration Lab

## Introduction

Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information about data and technology.  This information is called metadata.

Egeria provides servers to manage the exchange of metadata between different technologies. These servers are configured using REST API calls to an Open Metadata and Governance (OMAG) Server Platform.  Each call either defines a default value or configures a service that must run within the server when it is started.

As each configuration call is made, the server platform builds up a [configuration document](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/configuration-document.html) with the values passed.  When the configuration is finished, the configuration document will have all of the information needed to start the server.

The configuration document is deployed to the server platform that is hosting the server.  When a request is made to this server platform to start the server, it reads the configuration document and initializes the server with the appropriate services.

In this hands-on lab you will learn about the contents of configuration documents.

## The scenario

[Gary Geeke](https://opengovernance.odpi.org/coco-pharmaceuticals/personas/gary-geeke.html) is the IT Infrastructure leader at [Coco Pharmaceuticals](https://opengovernance.odpi.org/coco-pharmaceuticals/).

![Gary Geeke](https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/gary-geeke.png)

Gary's userId is `garygeeke`.

In [ ]:
adminUserId     = "garygeeke"

In the [Egeria Server Configuration](../egeria-server-config.ipynb) lab, Gary configured servers for the Open Metadata and Governance (OMAG) Server Platforms shown in Figure 1:

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

The following command checks that the platforms and servers are running.

In [ ]:
%run ../common/environment-check.ipynb

----
If the platform is not running, you will see a lot of red text. There are a number of choices on how to start it.  Follow [this link to set up and run the platform](https://egeria.odpi.org/open-metadata-resources/open-metadata-labs/).

Once the platform is running you are ready to proceed.

In this hands-on lab Gary is exploring the configuration document for the `cocoMDS1` server to understand how it is configured.  The cocoMDS1 server runs on the Data Lake OMAG Server Platform.

In [ ]:
mdrServerName = "cocoMDS1"
platformURLroot = dataLakePlatformURL

----

What follows are descriptions and coded requests to extract different parts of the configuration.

## Retrieve configuration for cocoMDS1 - Data Lake Operations metadata server

The command below retrieves the configuration document for `cocoMDS1`.  Its a big document so we will not display its full contents at this time.

In [ ]:

operationalServicesURLcore = "/open-metadata/admin-services/users/" + adminUserId

print (" ")
print ("Retrieving stored configuration document for " + mdrServerName + " ...")
url = platformURLroot + operationalServicesURLcore + '/servers/' + mdrServerName + '/configuration'
print ("GET " + url)

response = requests.get(url)

if response.status_code == 200:
    print("Server configuration for " + mdrServerName + " has been retrieved")
else:
    print("Server configuration for " + mdrServerName + " is unavailable")

serverConfig=response.json().get('omagserverConfig')

----
The configuration includes an audit trail that gives a high level overview of how the server has been configured.  This is always a useful starting point to understand the content of the configuration document for the server.

In [ ]:
auditTrail=serverConfig.get('auditTrail')

print (" ")

if auditTrail == None:
    print ("Empty configuration - no audit trail - configure the server before continuing")
else:    
    print ("Audit Trail: ")
    for x in range(len(auditTrail)): 
        print (auditTrail[x])

----
The rest of the lab notebook extracts the different sections from the configuration document and explains what they mean and how they are used in the server.

----
### Server names and identifiers

A server has a unique name that is used on all REST calls that concern it.  In addition, it is assigned a unique identifier (GUID) and an optional server type.  It is also possible to set up the name of the organization that owns the server.  These values are used in events the help locate the origin of metadata.

In [ ]:
print (" ")

serverName=serverConfig.get('localServerName')
if serverName != None:
    print ("Server name:   " + serverName)
    
serverGUID=serverConfig.get('localServerId')
if serverGUID != None:
    print ("Server GUID:   " + serverGUID)

serverType=serverConfig.get('localServerType')
if serverType != None:
    print ("Server Type:   " + serverType)

organization=serverConfig.get('organizationName')
if organization != None:
    print ("Organization:  " + organization)    
    

----
In addition, if the server has a local repository then the collection of metadata stored in it has a unique identifier (GUID) and a name.  These values are used to identify the origin of metadata instances since they are included in the audit header of any open metadata instance.

In [ ]:
print (" ")

repositoryServicesConfig = serverConfig.get('repositoryServicesConfig')
if repositoryServicesConfig != None:
    repositoryConfig = repositoryServicesConfig.get('localRepositoryConfig')
    if repositoryConfig != None:
        localMetadataCollectionId = repositoryConfig.get('metadataCollectionId')
        if localMetadataCollectionId != None:
             print ("Local metadata collection id:     " + localMetadataCollectionId)
        localMetadataCollectionName = repositoryConfig.get('metadataCollectionName')
        if localMetadataCollectionName != None:
             print ("Local metadata collection name:   " + localMetadataCollectionName)
      

----
Finally, a server with a repository that joins one or more cohorts needs to send out details of how a remote server should call this server during a federated query.  This information is called the **local repository's remote connection**.

By default, the network address that is defined in this connection begins with the value set in the **server URL root** property at the time the repository was configured. The server name is then added to the URL.

The code below extracts the server URL root and the **full URL endpoint** sent to other servers in the same cohort(s) in the local repository's remote connection.

In [ ]:
print (" ")

serverURLRoot=serverConfig.get('localServerURL')
if serverURLRoot != None:
    print ("Server URL root:   " + serverURLRoot)

if repositoryConfig != None:
    localRepositoryRemoteConnection = repositoryConfig.get('localRepositoryRemoteConnection')
    if localRepositoryRemoteConnection != None:
        endpoint = localRepositoryRemoteConnection.get('endpoint')
        if endpoint != None:
            fullURLEndpoint = endpoint.get('address')
            if fullURLEndpoint != None:
                print ("Full URL endpoint: " + fullURLEndpoint)

print (" ")

You will notice that the platform's specific network address is used in both values.

Using a specific network address is fine if the server is always going to run on this platform at this network address.  If the server is likely to be moved to a different platform, or the platform to a different location, it is easier to set up the full URL endpoint to include a logical DNS name.  This can be done by setting server URL root to this name before the local repository is configured, or updating the full URL endpoint in the local repository's remote connection.  When the repository next registers with the cohort, it will send out its new full URL endpoint as part of the registration request.

The complete local repository's remote connection is shown below.  Notice the **connectorProviderClassName** towards the bottom of the definition.  This is the factory class that creates the connector in the remote server.

In [ ]:
print (" ")
prettyResponse = json.dumps(localRepositoryRemoteConnection, indent=4)
print ("localRepositoryRemoteConnection: ")
print (prettyResponse)
print (" ")

----
The repository services running in a metadata repository uses a number of connectors to access the resources it needs.

The cocoMDS1 metadata server needs a local repository to store metadata about the data and processing occuring in the data lake.
This is the **local repository's remote connection**.

ODPi Egeria supports 2 types of repositories.  One is an in-memory repository that stores metadata in hash maps.  It is useful for demos and testing because a restart of the server results in an empty metadata repository.  However, if you need metadata to persist from one run of the server to the next, you should use the graph repository.

The code below shows which type of local repository is in use.  It also shows the destinations where audit log records are to be sent.  A server can have a list of destinations.  In this example, the server is just using a simple console log.


In [ ]:
print (" ")

if repositoryServicesConfig != None:
    auditLogConnections = repositoryServicesConfig.get('auditLogConnections')
    enterpriseAccessConfig = repositoryServicesConfig.get('enterpriseAccessConfig')
    cohortConfigList = repositoryServicesConfig.get('cohortConfigList')
    
if auditLogConnections != None:
    print ("Audit Log Destinations: ")
    for logDestCount in range(len(auditLogConnections)): 
        auditLogConnection = auditLogConnections[logDestCount]
        if auditLogConnection != None:
            connectorType = auditLogConnection.get('connectorType')
            if connectorType != None:
                description = connectorType.get('description')
                if description != None:
                    print (str(logDestCount+1) + ". description: " + description)
                connectorProviderClassName = connectorType.get('connectorProviderClassName')
                if connectorProviderClassName != None:
                    print ("   className:   " + connectorProviderClassName)
        
if repositoryConfig != None:
    localRepositoryLocalConnection = repositoryConfig.get('localRepositoryLocalConnection')

print (" ")

if localRepositoryLocalConnection != None:
    print ("Local Repository's Local Connection: ")
    connectorType = localRepositoryLocalConnection.get('connectorType')
    if connectorType != None:
        description = connectorType.get('description')
        if description != None:
            print ("  description: " + description)
        connectorProviderClassName = connectorType.get('connectorProviderClassName')
        if connectorProviderClassName != None:
            print ("  className:   " + connectorProviderClassName)


----

### Configuring security

There are two levels of security to set up for an ODPi Egeria server: authentication and authorization.


#### Authentication of servers and people

ODPi Egeria recommends that each server has its own identity and that is embedded with each request as part of the transport level security (TLS).  The members of the cohort (and the event topic) then grant access to each other and no-one else.

The identity of the calling user also flows with each request, but this time as a unique string value (typically userId) in the URL of the request.  You can see examples of this in the configuration requests being issued during this hands-on lab as Gary's userId `garygeeke` appears on each request.

The server configuration supports a userId and password for TLS.  The userId is also used when the server is processing requests that originate from an event and so there is no calling user.


In [ ]:
print (" ")

localServerUserId=serverConfig.get('localServerUserId')
if localServerUserId != None:
    print ("local Server UserId:   " + localServerUserId)
    
localServerPassword=serverConfig.get('localServerPassword')
if localServerPassword != None:
    print ("local Server Password: " + localServerPassword)


----
#### Authorization of metadata requests

ODPi Egeria servers also support a metadata security connector that plugs into the server and is called to provide authorization decisions as part of every request.

This connector is configured in the configuration document by passing the **Connection** object that provides the properties needed to create the connecto on the following call ...


In [ ]:
print (" ")

serverSecurityConnection=serverConfig.get('serverSecurityConnection')
if serverSecurityConnection != None:
    print ("Server's Security Connection:")
    prettyResponse = json.dumps(serverSecurityConnection, indent=4)
    print (prettyResponse)

print (" ")


----
### Setting up the event bus

The server needs to define the event bus it will use to exchange events about metadata.  This event bus configuration is used to connect to the cohorts and to provide the in / out topics for each of the Open Metadata Access Services (OMASs) - more later.

The event bus configuration for cocoMDS1 provides the network address that the event bus (Apache Kafka) is using.

In [ ]:

print (" ")

eventBusConfig=serverConfig.get('eventBusConfig')
if eventBusConfig != None:
    print ("Event Bus Configuration:")
    prettyResponse = json.dumps(eventBusConfig, indent=4)
    print (prettyResponse)

print (" ")


----
### Extracting the descriptions of the open metadata repository cohorts for the server

An open metadata repository cohort defines the servers that will share metadata.  A server can join multiple cohorts.  For
Coco Pharmaceuticals, cocoMDS1 is a member of the core `cocoCohort`.

![Figure 2](../images/coco-pharmaceuticals-systems-cohorts.png)
> **Figure 2:** Membership of Coco Pharmaceuticals' cohorts

You can see this in the configuration below.

In [ ]:
print (" ")

if cohortConfigList != None:
    print ("Cohort(s) that this server is a member of: ")
    for cohortCount in range(len(cohortConfigList)): 
        cohortConfig = cohortConfigList[cohortCount]
        if cohortConfig != None:
            cohortName = cohortConfig.get('cohortName')
            print (str(cohortCount+1) + ". name: " + cohortName)
            cohortRegistryConnection = cohortConfig.get('cohortRegistryConnection')
            if cohortRegistryConnection != None:
                print ("    Cohort Registry Connection: ")
                connectorType = cohortRegistryConnection.get('connectorType')
                if connectorType != None:
                    description = connectorType.get('description')
                    if description != None:
                        print ("     description: " + description)
                    connectorProviderClassName = connectorType.get('connectorProviderClassName')
                    if connectorProviderClassName != None:
                        print ("     className:   " + connectorProviderClassName)
            topicConnection = cohortConfig.get('cohortOMRSTopicConnection')
            if topicConnection != None:
                print ("    Cohort Topic Connection: ")
                connectorType = topicConnection.get('connectorType')
                if connectorType != None:
                    description = connectorType.get('description')
                    if description != None:
                        print ("     description: " + description)
                    connectorProviderClassName = connectorType.get('connectorProviderClassName')
                    if connectorProviderClassName != None:
                        print ("     className:   " + connectorProviderClassName)

----
### Reviewing the configured access services

Open Metadata Access Services (OMASs) provide the specialized APIs and events for specific tools and personas.  ODPi Egeria provides an initial set of access services, and additional services can be pluggable into the server platform.

To query the choice of access services available in the platform, use the follow command:

In [ ]:

print (" ")
print ("Retrieving the registered access services ...")
url = platformURLroot + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/registered-services/access-services"
print ("GET " + url)

response = requests.get(url)

prettyResponse = json.dumps(response.json(), indent=4)
print ("Response: ")
print (prettyResponse)
print (" ")


----
The `cocoMDS1` server is for the data lake operations.  It needs the access services to support the onboarding and decommissioning of assets along with the access services that supports the different engines that maintain the data lake.

In [ ]:
print (" ")

accessServiceConfig=serverConfig.get('accessServicesConfig')
if accessServiceConfig != None:
    print ("Configured Access Services: ")
    print (" ")
    for accessServiceCount in range(len(accessServiceConfig)): 
        accessServiceDefinition = accessServiceConfig[accessServiceCount]
        if accessServiceDefinition != None:
            accessServiceName = accessServiceDefinition.get('accessServiceName')
            accessServiceOptions = accessServiceDefinition.get('accessServiceOptions')
            if accessServiceName != None:
                print (" " + accessServiceName + " options: " + json.dumps(accessServiceOptions, indent=4))

print (" ")


----
### Listing the topics used by a server

Both the cohorts and the access services make extensive use of the event bus.  The code below extracts the names of all of the event bus topics used by this server.

In [ ]:
print (" ")
print ("List of Topics used by " + mdrServerName)


if cohortConfigList != None:
    for cohortCount in range(len(cohortConfigList)):
        cohortConfig = cohortConfigList[cohortCount]
        if cohortConfig != None:
            topicConnection = cohortConfig.get('cohortOMRSTopicConnection')
            if topicConnection != None:
                embeddedConnections = topicConnection.get('embeddedConnections')
                if embeddedConnections != None:
                    for connCount in range(len(embeddedConnections)):
                        embeddedConnection = embeddedConnections[connCount]
                        if embeddedConnection != None:
                            eventBusConnection = embeddedConnection.get('embeddedConnection')
                            if eventBusConnection != None:
                                endpoint = eventBusConnection.get('endpoint')
                                if endpoint != None:
                                    topicName = endpoint.get('address')
                                    if topicName != None:
                                        print ("  " + topicName)
    
    
if accessServiceConfig != None:
    for accessServiceCount in range(len(accessServiceConfig)): 
        accessService = accessServiceConfig[accessServiceCount]
        if accessService != None:
            eventBusConnection = accessService.get('accessServiceInTopic')
            if eventBusConnection != None:
                endpoint = eventBusConnection.get('endpoint')
                if endpoint != None:
                    topicName = endpoint.get('address')
                    if topicName != None:
                        print ("  " + topicName)
            eventBusConnection = accessService.get('accessServiceOutTopic')
            if eventBusConnection != None:
                endpoint = eventBusConnection.get('endpoint')
                if endpoint != None:
                    topicName = endpoint.get('address')
                    if topicName != None:
                        print ("  " + topicName)
        
            
print (" ")

----
### Controlling the volume of metadata exchange in a single REST call

To ensure that a caller can not request too much metadata in a single request, it is possible to set a maximum page size for requests that return a list of items.  The maximum page size puts a limit on the number of items that can be requested.  The variable below defines the value that will be added to the configuration document for each server.

In [ ]:
print (" ")

maxPageSize=serverConfig.get('maxPageSize')
if maxPageSize != None:
    print ("Maximum records return on a REST call: " + str(maxPageSize))
    

----
Finally, here is the configuration document in total

In [ ]:
print (" ")

prettyResponse = json.dumps(serverConfig, indent=4)
print ("Configuration for server: " + mdrServerName)
print (prettyResponse)
print (" ")

----